<a href="https://colab.research.google.com/github/clustering-jun/GNU-MachineLearning/blob/main/L8-3_%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8%EC%9D%98_%ED%8F%89%EA%B0%80%EC%99%80_%ED%99%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **분류 회귀 모델 사례: 타이타닉 데이터**

In [3]:
import pandas as pd

titanic = pd.read_csv('titanic.csv')
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [9]:
# 분류의 전처리 : 결측값 대체

print(titanic['Age'].mean())
print(titanic['Embarked'].value_counts())

29.69911764705882
Embarked
S    644
C    168
Q     77
Name: count, dtype: int64


In [11]:
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].mean())
titanic['Embarked'] = titanic['Embarked'].fillna('S')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [14]:
titanic.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [16]:
# 훈련 데이터와 테스트 데이터 분리, 연속형 독립변수와 범주형 독립변수 분리

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(titanic.drop(columns = 'Survived'),
                                                                 titanic['Survived'], test_size = 0.2,
                                                                 random_state=2025)

In [18]:
x_train_num = x_train[['Age', 'Fare', 'SibSp', 'Parch']]
x_test_num = x_test[['Age', 'Fare', 'SibSp', 'Parch']]

In [19]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train_num)

x_train_num_std = scaler.transform(x_train_num)
x_test_num_std = scaler.transform(x_test_num)

In [21]:
x_train_cat = x_train[['Sex', 'Embarked']]
x_test_cat = x_test[['Sex', 'Embarked']]

x_train_cat = pd.get_dummies(x_train_cat)
x_test_cat = pd.get_dummies(x_test_cat)

In [22]:
x_train_cat

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
166,True,False,False,False,True
563,False,True,False,False,True
197,False,True,False,False,True
606,False,True,False,False,True
313,False,True,False,False,True
...,...,...,...,...,...
504,True,False,False,False,True
588,False,True,False,False,True
862,True,False,False,False,True
734,False,True,False,False,True


In [29]:
x_train_f = pd.concat([pd.DataFrame(x_train_num_std, columns = ['Age', 'Fare', 'SibSp', 'Parch']),
                       x_train_cat.reset_index(drop=True)], axis = 1)
x_test_f = pd.concat([pd.DataFrame(x_test_num_std, columns = ['Age', 'Fare', 'SibSp', 'Parch']),
                       x_test_cat.reset_index(drop=True)], axis = 1)

x_train_f

,Age,Fare,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.397922,0.107353,0.0,0.166667,True,False,False,False,True
1,0.397922,0.015713,0.0,0.000000,False,True,False,False,True
2,0.565099,0.016404,0.0,0.166667,False,True,False,False,True
3,0.402011,0.015412,0.0,0.000000,False,True,False,False,True
4,0.374830,0.015412,0.0,0.000000,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...
707,0.211742,0.168837,0.0,0.000000,True,False,False,False,True
708,0.293286,0.015713,0.0,0.000000,False,True,False,False,True
709,0.646643,0.050610,0.0,0.000000,True,False,False,False,True
710,0.306877,0.025374,0.0,0.000000,False,True,False,False,True


In [30]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train_f, y_train)

RandomForestClassifier()

In [31]:
# 테스트 데이터를 이용하여 예측 성능 평가

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(x_test_f)

test_cm = confusion_matrix(y_test, y_pred)
test_acc = accuracy_score(y_test, y_pred)
test_prc = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

In [32]:
print(test_cm, test_acc, test_prc, test_recall, test_f1)

[[86 18]
 [27 48]] 0.7486033519553073 0.7272727272727273 0.64 0.6808510638297872


### **`cancer_classification.csv` 파일을 불러오고, 적절한 분류 모델을 사용하여 diagnosis에 대한 분류 모델을 학습시킨 후 모델의 성능평가를 진행하시오.
- [diagnosis (M = malignant, B = benign)]
- 데이터 설명 참고 링크: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data

In [35]:
import pandas as pd

cancer = pd.read_csv('cancer_classification.csv')
cancer

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [37]:
cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [40]:
cancer.drop(columns = ['id'], inplace=True)
cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [41]:
# 훈련 데이터와 테스트 데이터 분리, 연속형 독립변수와 범주형 독립변수 분리

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(cancer.drop(columns = 'diagnosis'),
                                                                 cancer['diagnosis'], test_size = 0.2,
                                                                 random_state=2025)

In [42]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train)

x_train_std = scaler.transform(x_train)
x_test_std = scaler.transform(x_test)

In [47]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

y_train

,M
526,False
397,False
392,True
38,True
524,False
...,...
568,False
160,False
211,False
323,True


In [64]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train_std, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [72]:
# 테스트 데이터를 이용하여 예측 성능 평가

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(x_test)

test_cm = confusion_matrix(y_test, y_pred)
test_acc = accuracy_score(y_test, y_pred)
test_prc = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [73]:
print("RandomForestClassifier ")
print("=" * 20)
print(f"Confusion Matrix:\n{test_cm}")
print("-" * 20)
print(f"Accuracy : {test_acc:.4f}")
print(f"Precision: {test_prc:.4f}")
print(f"Recall   : {test_recall:.4f}")
print(f"F1-score : {test_f1:.4f}")
print("=" * 20)

RandomForestClassifier 
Confusion Matrix:
[[ 0 68]
 [ 0 46]]
--------------------
Accuracy : 0.4035
Precision: 0.4035
Recall   : 1.0000
F1-score : 0.5750


In [74]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

model2 = DecisionTreeClassifier()
model2.fit(x_train_std, y_train)

y_pred2 = model2.predict(x_test)

test_cm = confusion_matrix(y_test, y_pred2)
test_acc = accuracy_score(y_test, y_pred2)
test_prc = precision_score(y_test, y_pred2)
test_recall = recall_score(y_test, y_pred2)
test_f1 = f1_score(y_test, y_pred2)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [75]:
print("DecisionTreeClassifier")
print("=" * 20)
print(f"Confusion Matrix:\n{test_cm}")
print("-" * 20)
print(f"Accuracy : {test_acc:.4f}")
print(f"Precision: {test_prc:.4f}")
print(f"Recall   : {test_recall:.4f}")
print(f"F1-score : {test_f1:.4f}")
print("=" * 20)

DecisionTreeClassifier
Confusion Matrix:
[[16 52]
 [ 1 45]]
--------------------
Accuracy : 0.5351
Precision: 0.4639
Recall   : 0.9783
F1-score : 0.6294
